In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np

f2 = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/val_data', 'rb')
X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
f2.close()

from keras.preprocessing.sequence import pad_sequences
from util import preprocess

total_X_sequence_train = []
total_X_feature_train = []
# total_X_image_train = []
total_Y_train = []
for i in range(0,10):
    f = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/train_data%d'%i, 'rb')
    X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f)
    X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
    X_sequence_train_trimed = preprocess(X_sequence_train_paded)
    X_feature_train = np.array(X_feature_train)
    Y_train = np.array(Y_train)-1

    Y_train = keras.utils.to_categorical(Y_train, num_classes=11)

    total_X_sequence_train.append(X_sequence_train_trimed)
    total_X_feature_train.append(X_feature_train)
    total_Y_train.append(Y_train)
    f.close()

X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_feature_val = np.array(X_feature_val)
Y_val = np.array(Y_val)-1


Y_val = keras.utils.to_categorical(Y_val, num_classes=11)

In [2]:
def rnn_with_feature(sequence_shape, feature_shape, num_classes):
    sequence_inputs = keras.Input(shape=sequence_shape, name='sequence')
    masking_layer = layers.Masking(mask_value=0)
    x1 = masking_layer(sequence_inputs)
    x1 = layers.Bidirectional(layers.GRU(128, return_sequences=True))(x1)
    x1 = layers.Bidirectional(layers.GRU(64))(x1)
    x1 = layers.Dropout(0.4)(x1)
    x1 = layers.Dense(128, activation="relu")(x1)
    x1 = layers.Dropout(0.4)(x1)
    x1 = layers.Dense(32, activation="relu")(x1)

    feature_inputs = keras.Input(shape=feature_shape, name='feature')
    x3 = feature_inputs
    x3 = layers.Dense(64, activation="relu")(x3)
    x3 = layers.Dense(32, activation="relu")(x3)

    x = layers.concatenate([x1, x3])

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs=[sequence_inputs, feature_inputs], outputs=outputs)
    return model

In [4]:
X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=430,
    decay_rate=0.9)

for i in range(1,10):
    model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
    model.compile(
    optimizer=keras.optimizers.Adam(lr_schedule),
    # loss="sparse_categorical_crossentropy",
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
    )
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='models/bagging/rnn_with_feature%d/model_{epoch}'%i,
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='models/bagging/rnn_with_feature%d/logs'%i)
    ]
    X_train = {'sequence':total_X_sequence_train[i], 'feature':total_X_feature_train[i]}
    model.fit(X_train, total_Y_train[i], batch_size=32, epochs=7, 
        validation_data=(X_val, Y_val), callbacks=callbacks)


2022-10-19 16:06:46.048121: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-19 16:06:46.048153: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-10-19 16:06:46.603604: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-19 16:06:46.603771: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


Epoch 1/7
  1/860 [..............................] - ETA: 1:52:18 - loss: 3.4122 - acc: 0.0625

2022-10-19 16:06:54.927974: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-10-19 16:06:54.928013: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/860 [..............................] - ETA: 12:09 - loss: 5.0656 - acc: 0.1406  

2022-10-19 16:06:55.667668: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-10-19 16:06:55.668032: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-10-19 16:06:55.718323: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 5505 callback api events and 5502 activity events. 
2022-10-19 16:06:55.797401: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-10-19 16:06:55.895228: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: models/bagging/rnn_with_feature1/logs/train/plugins/profile/2022_10_19_16_06_55

2022-10-19 16:06:55.964688: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to models/bagging/rnn_with_feature1/logs/train/plugins/profile/2022_10_19_16_06_55/dell-Precision-7920-Tower.trace.json.gz
2022-10-19 16:06:56.026614: I tensorflow/

...

INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_6/assets


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_6/assets


Epoch 7/7
860/860 [==============================] - 85s 99ms/step - loss: 0.8258 - acc: 0.8909 - val_loss: 0.9571 - val_acc: 0.8203


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_7/assets


INFO:tensorflow:Assets written to: models/bagging/rnn_with_feature9/model_7/assets
